In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# accelerator 加速装置
# available 可利用的
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

print(f"Using {device} device")
print(torch.accelerator.is_available())


Using cpu device
False


验证设备是否存在加速装置，如果存在就利用他否则则使用CPU。

In [3]:
# neural 神经的
# network 网络

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()  # 使平坦
        self.linear_relu_stack = nn.Sequential(  # 直线的， 激活函数， 堆叠， 按次序的
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")


Predicted class: tensor([4])


将图片张量输入模型，模型输出一个二维张量。每一行表示一个批次内不同的样本，每一列表示不同的类别。从每一行中选出概率最大的那个类别。也就是模型的预测结果。

In [11]:
# 模型曾

input_image = torch.rand(3, 28, 28)
print(input_image.size())


torch.Size([3, 28, 28])


In [35]:
# 利用nn.Flatten扁平化展开除每一批次的样本的张量。

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())  # 展开后的形状 [3, 748]

print("-" * 70)

tensor = torch.tensor([
    [
        [
            [0, 0, 255],
            [0, 255, 255],
            [255, 0, 255],
        ],
        [
            [0, 255, 0],
            [255, 0, 255],
            [255, 255, 255]
        ],
        [
            [1, 5, 5],
            [3, 4, 7],
            [7, 8, 10]
        ]
    ],
    [
        [
            [255, 0, 100],
            [150, 255, 190],
            [100, 30, 255],
        ],
        [
            [2, 4, 5],
            [255, 150, 155],
            [190, 155, 255]
        ],
        [
            [2, 4, 7],
            [4, 7, 8],
            [255, 250, 200]
        ]
    ],
    [
        [
            [12, 155, 255],
            [255, 255, 255],
            [150, 150, 70]
        ],
        [
            [240, 255, 100],
            [10, 20, 30],
            [255, 255, 255]
        ],
        [
            [255, 255, 255],
            [250, 250, 250],
            [150, 250, 245]
        ]
    ]
])

print(tensor.size())
flat_tensor = flatten(tensor)
print(flat_tensor.size())
print()
print(flat_tensor)



torch.Size([3, 784])
----------------------------------------------------------------------
torch.Size([3, 3, 3, 3])
torch.Size([3, 27])

tensor([[  0,   0, 255,   0, 255, 255, 255,   0, 255,   0, 255,   0, 255,   0,
         255, 255, 255, 255,   1,   5,   5,   3,   4,   7,   7,   8,  10],
        [255,   0, 100, 150, 255, 190, 100,  30, 255,   2,   4,   5, 255, 150,
         155, 190, 155, 255,   2,   4,   7,   4,   7,   8, 255, 250, 200],
        [ 12, 155, 255, 255, 255, 255, 150, 150,  70, 240, 255, 100,  10,  20,
          30, 255, 255, 255, 255, 255, 255, 250, 250, 250, 150, 250, 245]])


通过Flatten将高维的张量进行展平，但是批次通道不变。
方法： from torch import nn flatten = nn.Flatten(), 然后输入图片张量进行展开 flat_image = flatten(input_image).

In [36]:
# nn.Linear  线性
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())


torch.Size([3, 20])


对展开的张量进行线性变换。in_features 和 out_features 参数分别代表输入全连接层的元素数量，也就是展开之后每个样本的一维张量里的元素数量。

In [37]:
# nn.ReLU  激活函数

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU(hidden1)
print(f"After ReLU: {hidden1}")



Before ReLU: tensor([[ 0.2243,  0.2623, -0.2424, -0.1337,  0.3282,  0.1164,  0.2405, -0.4279,
         -0.0632,  0.1972, -0.5300, -0.4372, -0.0975, -0.5016, -0.2576, -0.2612,
          0.1572, -0.4781,  0.5032,  0.0940],
        [-0.0807, -0.0910,  0.0224, -0.1962,  0.1052,  0.3374,  0.1149, -0.2441,
          0.2381,  0.1248, -0.1424, -0.4403,  0.2714,  0.0318, -0.4273, -0.2490,
          0.6695, -0.4543, -0.1088, -0.3009],
        [ 0.0903,  0.1500,  0.0014, -0.1499,  0.0213,  0.6564,  0.1761, -0.0669,
          0.2026,  0.2525,  0.0395, -0.3136, -0.0528,  0.0408, -0.2764, -0.1258,
          0.4835, -0.4795,  0.0207, -0.1260]], grad_fn=<AddmmBackward0>)




RuntimeError: Boolean value of Tensor with more than one value is ambiguous

通过激活函数引入非线性映射，让神经网络学习到各种各样通用的特征，而非记住参数和顺序。
方法： out = nn.ReLU(Linear_in)